In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [4]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_in.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,IN,46327.0,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,IN,46204.0,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,IN,46268.0,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,IN,46406.0,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,IN,46573.0,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,IN,46268.0,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,1.809731e+14
16597,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,IN,46107.0,...,999 Other,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,1.809738e+14
16598,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,IN,46947.0,...,999 Other,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,1.801795e+14
16599,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,IN,46033.0,...,999 Other,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,1.805711e+14


In [5]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [6]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
1256,6612367104,"PO Box 125, LAMAR, IN",NaN,NaN
2198,9144177010,"4211 CR 61 0.0, Butler, IN",NaN,NaN
2544,4965218206,"PO BOX 119, CLIFFORD, IN",NaN,NaN
6037,2346618406,"4 W Plymouth St, Hamlet, IN",NaN,NaN
6038,4585587105,"4 W. Plymouth St. PO Box 390, HAMLET, IN",NaN,NaN
7495,4742738408,"3511 W Napoleon Main St, Napoleon, IN",NaN,NaN
7496,6157397110,"3511 W Napoleon Main St, NAPOLEON, IN",NaN,NaN
9016,1519617110,"P O Box 156, BUTLER, IN",NaN,NaN
9774,5680117009,"PO BOX 201 23326, SHELBY, IN",NaN,NaN
10421,6106497007,"PO BOX 65, LEO, IN",NaN,NaN


In [7]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1266777200,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830
1,2313017204,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000
2,2341197209,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910
3,3129227104,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965
4,5372347006,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [7]:
fips_check.to_csv("../data/state_data/geo/geo_fips/IN_fips_scraped.csv")

In [9]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/IN_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,1,1,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2,2,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3,3,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,4,4,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14


In [10]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [11]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [12]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [13]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1266777200,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,2313017204,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2341197209,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3129227104,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,5372347006,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14
...,...,...,...,...,...
16596,9384688310,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,1.809731e+14
16597,9490728302,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,1.809738e+14
16598,9571197108,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,1.801795e+14
16599,9715977705,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,1.805711e+14


In [16]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() #35

LoanNumber    35
full_add      35
Lat           35
Long          35
FIPS_z         0
dtype: int64

<h4>FIPS Script

In [17]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [30]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)

option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('18')] #IN
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

The version of chrome cannot be detected. Trying with latest driver version


  0%|          | 0/35 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [31]:
geo_id_list =geo_id 

Length Check

In [32]:
print(len(geo_id_list))

35


Matching FIPS to DataFrame

In [34]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null.iloc[:35]

In [35]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
260,3269267107,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,[]
364,7007407110,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,[]
412,6311077009,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,[]
581,5283147003,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,[]
1022,7535037110,"701 WATER ST, FREMONT, IN",40.624320,-79.153366,[]
1370,6565827009,"P.O Box 67 425 Main Street NE, PALMYRA, IN",40.758663,-73.953093,[]
2092,3646578403,"5401 W Airport Rd, Gary, IN",40.653263,-79.039024,[]
2277,1201748502,"111 E Innovation Blvd, Syracuse, IN",40.621834,-79.139520,[]
2585,3344197100,"5401 AIRPORT RD, GARY, IN",40.653255,-79.039030,[]
2811,4919698100,"1531 13th St. Suite G900, NASHVILLE, IN",36.162664,-86.781602,[]


Remove brackets

In [36]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
260,3269267107,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,7007407110,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,6311077009,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
581,5283147003,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
1022,7535037110,"701 WATER ST, FREMONT, IN",40.624320,-79.153366,NaN
1370,6565827009,"P.O Box 67 425 Main Street NE, PALMYRA, IN",40.758663,-73.953093,NaN
2092,3646578403,"5401 W Airport Rd, Gary, IN",40.653263,-79.039024,NaN
2277,1201748502,"111 E Innovation Blvd, Syracuse, IN",40.621834,-79.139520,NaN
2585,3344197100,"5401 AIRPORT RD, GARY, IN",40.653255,-79.039030,NaN
2811,4919698100,"1531 13th St. Suite G900, NASHVILLE, IN",36.162664,-86.781602,NaN


In [37]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
260,3269267107,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,7007407110,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,6311077009,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
581,5283147003,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
1022,7535037110,"701 WATER ST, FREMONT, IN",40.624320,-79.153366,NaN
1370,6565827009,"P.O Box 67 425 Main Street NE, PALMYRA, IN",40.758663,-73.953093,NaN
2092,3646578403,"5401 W Airport Rd, Gary, IN",40.653263,-79.039024,NaN
2277,1201748502,"111 E Innovation Blvd, Syracuse, IN",40.621834,-79.139520,NaN
2585,3344197100,"5401 AIRPORT RD, GARY, IN",40.653255,-79.039030,NaN
2811,4919698100,"1531 13th St. Suite G900, NASHVILLE, IN",36.162664,-86.781602,NaN


Parse for Merge

In [22]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
5,5434877008,187916949452020
17,1683977300,181270501042051
18,6084567002,180699617004009
23,9771547100,181410115051003
28,9116507201,181379685001004
...,...,...
16562,6309287101,180390019021006
16567,6929428604,180973608003004
16578,7766637110,180910418004032
16580,7921298308,180571105163086


Read in base data

In [23]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_in.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/IN_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,1,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,4,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14


Merge with MainFrame

In [24]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14,NaN
1,1,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14,NaN
2,2,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14,NaN
3,3,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14,NaN
4,4,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,16596,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,...,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,1.809731e+14,NaN
16597,16597,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,...,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,1.809738e+14,NaN
16598,16598,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,...,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,1.801795e+14,NaN
16599,16599,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,...,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,1.805711e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [25]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14,NaN,180890204001028.0
1,1,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14,NaN,180973910011101.0
2,2,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14,NaN,180973102011027.0
3,3,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14,NaN,180890103023014.0
4,4,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14,NaN,180390013011018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,16596,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,...,5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,1.809731e+14,NaN,180973101051006.0
16597,16597,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,...,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,1.809738e+14,NaN,180973804042001.0
16598,16598,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,...,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,1.801795e+14,NaN,180179517001009.0
16599,16599,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,...,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,1.805711e+14,NaN,180571110033009.0


Drop Extra FIPS Columns

In [26]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,180890204001028.0
1,1,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,180973910011101.0
2,2,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,180973102011027.0
3,3,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,180890103023014.0
4,4,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,180390013011018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,16596,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,180973101051006.0
16597,16597,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,...,999 Other,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,180973804042001.0
16598,16598,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,...,999 Other,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,180179517001009.0
16599,16599,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,...,999 Other,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,180571110033009.0


In [27]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
260,260,350992,350992,3269267107,4/11/20,562,PPP,"INDIANAPOLIS ZOOLOGICAL SOCIETY, INC",1200 W Washington Street 0.0,Indianapolis,...,999 Other,"7121 Museums, Historical Sites, and Similar In...",7121.0,INDIANA DISTRICT OFFICE,d $2-5 million,250 to 499,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,364,351096,351096,7007407110,4/14/20,562,PPP,FAB9 INC,9269 E STATE ROAD 48,MILAN,...,999 Other,4841 General Freight Trucking,4841.0,INDIANA DISTRICT OFFICE,d $2-5 million,100 to 249,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,412,351144,351144,6311077009,4/6/20,562,PPP,"PIKE LUMBER COMPANY, INC.",719 FRONT ST,AKRON,...,321 Wood Products,3211 Sawmills and Wood Preservation,3211.0,INDIANA DISTRICT OFFICE,d $2-5 million,100 to 249,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
509,509,351241,351241,8772907000,4/8/20,562,PPP,"VALLEY PROFESSIONALS COMMUNITY HEALTH CENTER, ...",777 South Main Street Suite 100,CLINTON,...,999 Other,6213 Offices of Other Health Practitioners,6213.0,INDIANA DISTRICT OFFICE,d $2-5 million,100 to 249,"777 South Main Street Suite 100, CLINTON, IN",NaN,NaN,NaN
581,581,351313,351313,5283147003,4/5/20,562,PPP,"PROCESS DEVELOPMENT & FABRICATION, INC.",PO Box 493,BRAZIL,...,332 Fabricated Metal Products,3323 Architectural and Structural Metals Manuf...,3323.0,INDIANA DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15668,15668,366400,366400,4653648305,1/23/21,562,PPS,COMPLETE MAINTENANCE SERVICES INC,200 S Parker Dr,Ashley,...,332 Fabricated Metal Products,"3328 Coating, Engraving, Heat Treating, and Al...",3328.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"200 S Parker Dr, Ashley, IN",NaN,NaN,NaN
15964,15964,366696,366696,6045077005,4/6/20,562,PPP,"CAMRISTONE INTERNAIONAL, INC",10525 W US Hwy 30 Bldg 11 B,WANATAH,...,999 Other,4239 Miscellaneous Durable Goods Merchant Whol...,4239.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"10525 W US Hwy 30 Bldg 11 B, WANATAH, IN",40.778572,-98.740866,NaN
15978,15978,366710,366710,8056608402,2/12/21,562,PPS,COMPLEMENTARY FAMILY MEDICAL CARE LLC,69 E Garner Rd,Brownsburg,...,999 Other,6211 Offices of Physicians,6211.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"69 E Garner Rd, Brownsburg, IN",40.622322,-79.139510,NaN
16451,16451,367183,367183,8460197100,4/15/20,562,PPP,"R.P. WAKEFIELD CO., INC.",600 W Maple Street PO Box 97,WATERLOO,...,335 Electrical Equipment,3359 Other Electrical Equipment and Component ...,3359.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"600 W Maple Street PO Box 97, WATERLOO, IN",40.613804,-79.150734,NaN


Convert to File

In [28]:
fips_IN = fips_merge_drop
#fips_TN = fips_merge_drop

In [29]:
fips_IN.to_csv("../../data/state_data/geo/geo_fips/IN_fips_scraped.csv") 

Review & Compare

In [30]:
fips_IN = pd.read_csv("../../data/state_data/geo/geo_fips/IN_fips_scraped.csv") 
fips_IN

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,1,1,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2,2,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3,3,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,4,4,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,16596,16596,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,1.809731e+14
16597,16597,16597,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,...,999 Other,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,1.809738e+14
16598,16598,16598,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,...,999 Other,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,1.801795e+14
16599,16599,16599,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,...,999 Other,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,1.805711e+14
